In [1]:
#! pip install beautifulsoup4, requests
# !pip install selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
import os
import sys
os.path.dirname(sys.executable)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time

In [16]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI"
html = requests.get(url)
html.content[0:500]

b'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel='

In [17]:
# Making the soup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html.content, 'lxml')

In [29]:
#soup.head()

In [26]:
# Identifying the restaurant links
links = soup.select(".css-166la90")
print(links[0:10])

# Getting the restaurant names and the restaurant links in solation
name_restaurant = []
link_restaurant = []

for link in links:
    name_restaurant.append(link.string)
    link_restaurant.append(link['href'])
    
print(name_restaurant[0:5])

[<a class="css-166la90" href="/biz/swingin-door-exchange-milwaukee?osq=Restaurants" name="Swingin’ Door Exchange" rel="" target="">Swingin’ Door Exchange</a>, <a class="css-166la90" href="/biz/story-hill-bkc-milwaukee?osq=Restaurants" name="Story Hill BKC" rel="" target="">Story Hill BKC</a>, <a class="css-166la90" href="/biz/maxies-milwaukee?osq=Restaurants" name="Maxie’s" rel="" target="">Maxie’s</a>, <a class="css-166la90" href="/biz/triciclo-peru-milwaukee?osq=Restaurants" name="Triciclo Peru" rel="" target="">Triciclo Peru</a>, <a class="css-166la90" href="/biz/tusk-milwaukee-2?osq=Restaurants" name="Tusk" rel="" target="">Tusk</a>, <a class="css-166la90" href="/biz/view-mke-milwaukee?osq=Restaurants" name="View MKE" rel="" target="">View MKE</a>, <a class="css-166la90" href="/biz/mad-rooster-cafe-west-milwaukee?osq=Restaurants" name="Mad Rooster Cafe" rel="" target="">Mad Rooster Cafe</a>, <a class="css-166la90" href="/biz/st-paul-fish-company-milwaukee?osq=Restaurants" name="St 

In [ ]:
# Getting everything also for multiple pages (here, for page 2, 3, 4, 5, and 6)
url_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start="

for i in range(0,240,10):
    
    html = requests.get(url_yelp+str(i))
    
    soup = BeautifulSoup(html.content, 'lxml')
    link = soup.select(".css-166la90")
   
    for link in links:
        name_restaurant.append(link.string)
        link_restaurant.append(link.get('href'))

# Making sure we have everything
print(name_restaurant)  
print(link_restaurant)

In [50]:
#let's zip the lists up to be abele to convert them into a dataframe
data_tuples = list(zip(name_restaurant,link_restaurant))
len(data_tuples)

418

In [51]:
#creating a dataframe with two columns, this one will later be enriched by all the data we scrape
df = pd.DataFrame(data_tuples, columns=['Restaurant', 'Links'])

In [52]:
# inpsecting the dataframe, we can see that beautiful soup has extracted some incorrect numbers we don't need, and we still need to clean up the links
df.tail(10)

,Restaurant,Links
408,El Beso Milwaukee - Mexican Restaurante & Cantina,/biz/el-beso-milwaukee-mexican-restaurante-and...
409,None,https://www.yelp.com/search?find_desc=Restaura...
410,16,https://www.yelp.com/search?find_desc=Restaura...
411,17,https://www.yelp.com/search?find_desc=Restaura...
412,18,https://www.yelp.com/search?find_desc=Restaura...
413,19,https://www.yelp.com/search?find_desc=Restaura...
414,20,https://www.yelp.com/search?find_desc=Restaura...
415,21,https://www.yelp.com/search?find_desc=Restaura...
416,22,https://www.yelp.com/search?find_desc=Restaura...
417,23,https://www.yelp.com/search?find_desc=Restaura...


In [46]:
#let's us regex to clean the df
import re

In [53]:
#we convert everyting to strings, and then exclude those rows where the column Links contains a https
df['Restaurant'] = df.Restaurant.astype('str') 
df['Links'] = df.Links.astype('str') 

df=df[~df.Links.str.contains("https:")]

In [54]:
#then we complete the links colum for the full URL
df['Complete_links']= df['Links'].apply(lambda x:"https://www.yelp.com"+str(x))

In [57]:
df.pop('Links')

0      /biz/swingin-door-exchange-milwaukee?osq=Resta...
1          /biz/story-hill-bkc-milwaukee?osq=Restaurants
2                  /biz/maxies-milwaukee?osq=Restaurants
3           /biz/triciclo-peru-milwaukee?osq=Restaurants
4                  /biz/tusk-milwaukee-2?osq=Restaurants
                             ...                        
404    /biz/la-fondita-de-milwaukee-milwaukee?osq=Res...
405    /biz/taqueria-buenavista-milwaukee?osq=Restaur...
406    /biz/michaels-family-restaurant-milwaukee-2?os...
407             /biz/triskeles-milwaukee?osq=Restaurants
408    /biz/el-beso-milwaukee-mexican-restaurante-and...
Name: Links, Length: 210, dtype: object

In [55]:
len(df) 

210

In [58]:
df.tail(10)

,Restaurant,Complete_links
399,Stone Bowl Grill,https://www.yelp.com/biz/stone-bowl-grill-milw...
400,Dugout 54,https://www.yelp.com/biz/dugout-54-milwaukee?o...
401,Flour Girl and Flame,https://www.yelp.com/biz/flour-girl-and-flame-...
402,Snifters Tapas & Spirits,https://www.yelp.com/biz/snifters-tapas-and-sp...
403,Bartolotta’s Lake Park Bistro,https://www.yelp.com/biz/bartolottas-lake-park...
404,La Fondita De Milwaukee,https://www.yelp.com/biz/la-fondita-de-milwauk...
405,Taqueria Buenavista,https://www.yelp.com/biz/taqueria-buenavista-m...
406,Michael’s Family Restaurant,https://www.yelp.com/biz/michaels-family-resta...
407,Triskele’s,https://www.yelp.com/biz/triskeles-milwaukee?o...
408,El Beso Milwaukee - Mexican Restaurante & Cantina,https://www.yelp.com/biz/el-beso-milwaukee-mex...


In [70]:
# we see that there are two duplicate restaurants, which we can drop subsequnetly
df.groupby(['Restaurant']).size().sort_values(ascending=False)

Restaurant
Egg & Flour Pasta Bar     2
Camino                    2
2Mesa Mexican Eatery      1
Shake Shack               1
Riley’s Sandwich Co       1
                         ..
Goodkind                  1
Guadalajara Restaurant    1
Hamburger Mary’s          1
Harbor House              1
Zarletti                  1
Length: 208, dtype: int64

In [76]:
df.drop_duplicates(keep='first', inplace=True)

In [78]:
df.to_csv('links.csv',index=False,header=True,encoding='utf8')

### Next, we scrape all the details about the restaurants with a web driver and the selenium package

In [2]:
df = pd.read_csv('links.csv')

In [3]:
df.head()

,Restaurant,Complete_links
0,Swingin’ Door Exchange,https://www.yelp.com/biz/swingin-door-exchange...
1,Story Hill BKC,https://www.yelp.com/biz/story-hill-bkc-milwau...
2,Maxie’s,https://www.yelp.com/biz/maxies-milwaukee?osq=...
3,Triciclo Peru,https://www.yelp.com/biz/triciclo-peru-milwauk...
4,Tusk,https://www.yelp.com/biz/tusk-milwaukee-2?osq=...


In [4]:
'''options to avoide that actual windows are opened every time. create headless driver options'''
options = webdriver.FirefoxOptions()

#these options are set so that a browser doesn't actually open everytime we scrape
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [5]:
Links = df.Complete_links.to_list()
Restaurants = df.Restaurant.to_list()

In [764]:
# we make a little test run with just one restaurant
l=Links[1:4]
r = Restaurants[1:4]

In [765]:
print(l,r)

['https://www.yelp.com/biz/story-hill-bkc-milwaukee?osq=Restaurants', 'https://www.yelp.com/biz/maxies-milwaukee?osq=Restaurants', 'https://www.yelp.com/biz/triciclo-peru-milwaukee?osq=Restaurants'] ['Story Hill BKC', 'Maxie’s', 'Triciclo Peru']


In [52]:
'''Selenium search automation can either fetch the search item from the list or text can be entered'''
def searchrestaurant(Links, Restaurants):
    for i in Links:
        for j in Restaurants:
            #startdriver
            #time.sleep(5)
            #driver = webdriver.Firefox(options=options) #options=options
            html = requests.get(i)
            
        
            soup = BeautifulSoup(html.content, 'lxml')
            description = soup.select(".css-gdi06s")
            numberofreviews = soup.select(".css-bq71j2") 
            photos = soup.select(".css-ardur")
            rating = soup.select(".i-stars__373c0__1T6rz i-stars--large-4-half__373c0__2lYkD border-color--default__373c0__30oMI overflow--hidden__373c0__2B0kz")
            
        return description, numberofreviews, photos, rating
    

        ##we should have lists with the name, and then a list for each colum we would like to extract

In [363]:
soup.find_all(class_ = ' margin-b2__373c0__abANL border-color--default__373c0__2oFDT')[1].findChildren(class_ = ' css-1h1j0y3')

['Story Hill BKC', 'Maxie’s', 'Triciclo Peru']

In [ ]:
'''Selenium search automation can either fetch the search item from the list or text can be entered'''

df_rest = pd.DataFrame([],columns=['restaurant', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])


#This is so we can enumerate over the restaurant names
enum_extract = enumerate(Restaurants)

def getname():
    key = next(enum_extract)
    return str(key)


for i in Links:
    restaurant_update = []
    restaurant_description = []
    restaurant_numberofreviews = []        
    restaurant_photos = []
    restaurant_rating = []
    restaurant_street = []
    restaurant_adress = []

    restaurant_link = []
    restaurant_price = []
    restaurant_categories = []
    restaurant_name = [] 

    restaurant_claimed = [] 
    restaurant_amenities = []

    #so here we append the restaurant name
    restname = getname()
    restaurant_name.append(restname)  
    
    try:
            #Here we initialize a headless driver
        time.sleep(3)
        driver = webdriver.Firefox(options=options) #options=options 
        driver.get(i)

        driver.maximize_window()
        time.sleep(3)

        #we first need to get rid of the privacy concern button so we click it
        driver.find_element_by_css_selector('.onetrust-lg').click()

        #then we wait until that click is processed, before we click the button where all the amenties are extended
        time.sleep(3)
        driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()

        source = driver.page_source
        soup1 = BeautifulSoup(source, 'html.parser')


        #now we loop over the elements in the class
        subsoup = soup1.find_all(class_ = 'arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT')
        amenities = []
        for k in range(len(subsoup)):
            subamenities=subsoup[k].get_text()
            amenities.append(subamenities)

        restaurant_amenities.append(amenities) 


        #number of reviews
        numberofreviews = soup1.select(".css-bq71j2") 
        restaurant_numberofreviews.append(numberofreviews[0].string)

        #number of photos
        photos = soup1.select(".css-ardur")
        restaurant_photos.append(photos[4].string) 

        #first and second address line
        location = soup1.select(".raw__373c0__3rcx7")
        restaurant_street.append(location[0].string) 
        restaurant_adress.append(location[1].string)

            #rating
        for item in soup1.select('[class*=container]'):
            if item.find('h4'):
                name = item.find('h4').get_text()
                ratings=soup1.select('[aria-label*=rating]')[0]
                rating=str(ratings)
                rating = re.search('aria-label="(.+?)star rating"', rating)
                if rating:
                    rating = rating.group(1)

        restaurant_rating.append(rating)

        #price
        price = soup1.select(".css-1xxismk")
        price=str(price)
        price = re.search('"css-1xxismk">(.+?)<', price)
        if price:
            price = price.group(1)
            restaurant_price.append(price) 

        #categories
        #this list needs to be inside the loop
        categories = []

        for item in soup1.select('[class*=container]'):
            if item.find('h1'):
                items=item.select('.css-166la90')
                for i in items:
                    categories.append(i.string)

                restaurant_categories.append(categories)

        if 'Unclaimed' not in categories:

            #updates
            update=soup1.select('.css-gdi06s')
            restaurant_update.append(update[0].string)             


            #homepage link
            link = soup1.select(".css-ac8spe")
            restaurant_link.append(link[6].string)


            restaurant_claimed.append('Claimed')

        else: 
            restaurant_update.append('None')
            restaurant_link.append('None')
            restaurant_claimed.append('Unclaimed')

            driver.quit()


    except: 
        restaurant_claimed.append('None')
        restaurant_update.append('None')
        restaurant_numberofreviews.append('None')
        restaurant_photos.append('None')
        restaurant_rating.append('None')
        restaurant_street.append('None')
        restaurant_adress.append('None')
        restaurant_link.append('None')
        restaurant_price.append('None')
        restaurant_categories.append('None')
        restaurant_amenities.append('None')
            
    finally:
        if len(restaurant_claimed) == 0:
            restaurant_claimed.append('None')
                
        if len(restaurant_update) == 0:
            restaurant_update.append('None')
                
        if len(restaurant_numberofreviews) == 0:
            restaurant_numberofreviews.append('None')
                
        if len(restaurant_photos) == 0:
            restaurant_photos.append('None')
                
        if len(restaurant_rating) == 0:
            restaurant_rating.append('None')
                
        if len(restaurant_street) == 0:
            restaurant_street.append('None')
                
        if len(restaurant_adress) == 0:
            restaurant_adress.append('None')
                
        if len(restaurant_link) == 0:
            restaurant_link.append('None')
                
        if len(restaurant_price) == 0:
            restaurant_price.append('None')
                
        if len(restaurant_categories) == 0:
            restaurant_categories.append('None')
                
        if len(restaurant_amenities) == 0:
            restaurant_amenities.append('None')
                
        entry_list = list(zip(restaurant_name, restaurant_claimed, restaurant_numberofreviews, restaurant_photos, restaurant_price, restaurant_rating, restaurant_categories, restaurant_street, restaurant_adress, restaurant_link,restaurant_update, restaurant_amenities))

            
        df_entry = pd.DataFrame(entry_list,columns=['restaurant', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])
            
        frames = [df_rest, df_entry]
        df_rest = pd.concat(frames)       

In [15]:
df_rest

,restaurant,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
0,"(0, 'Triciclo Peru')",Unclaimed,39 reviews,See 64 photos,$,4.5,"[Unclaimed, Breweries, Beer Bar, Beer Gardens]",1024 S 5th St,"Milwaukee, WI 53204",None,None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(1, 'Tusk')",Claimed,67 reviews,See 125 photos,None,4.5,[Peruvian],3801 W Vliet St,"Milwaukee, WI 53208",tricicloperumke.com,None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(2, 'View MKE')",Claimed,57 reviews,See 120 photos,$$,4.5,"[American (New), Bars]",5513 W North Ave,"Milwaukee, WI 53208",tuskmilwaukee.com,None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(3, 'Mad Rooster Cafe')",None,None,None,None,None,None,None,None,None,None,None


In [12]:
R = Restaurants[3:7]
z = Links[3:6]
L = ['https://www.yelp.com/biz/urban-harvest-brewing-company-milwaukee'] + z

In [13]:
R, L

(['Triciclo Peru', 'Tusk', 'View MKE', 'Mad Rooster Cafe'],
 ['https://www.yelp.com/biz/urban-harvest-brewing-company-milwaukee',
  'https://www.yelp.com/biz/triciclo-peru-milwaukee?osq=Restaurants',
  'https://www.yelp.com/biz/tusk-milwaukee-2?osq=Restaurants',
  'https://www.yelp.com/biz/view-mke-milwaukee?osq=Restaurants'])

In [811]:
name = []

In [812]:
for r in R:
    name.append(r)

In [813]:
name

['Triciclo Peru', 'Tusk', 'View MKE', 'Mad Rooster Cafe']

In [786]:
description=soup2.select('.css-gdi06s')

In [787]:
restaurant_description = []
restaurant_description.append(description[0].string)
restaurant_description

['"The same Movida food you love, same great cocktails, with all the sangria you can enjoy. This is Movida at Hotel Madrid.\n\nMovida, translated meaning "the movement", is a Spanish tapas restaurant located in Milwaukee, Wisconsin. Beyond the food, Movida features a recognize…"']

In [794]:
restaurant_numberofreviews.append('none')

In [795]:
restaurant_numberofreviews

['39 reviews', 'none']

In [780]:
restaurant_numberofreviews=[]
numberofreviews = soup8.select(".css-bq71j2") 
restaurant_numberofreviews.append(numberofreviews[0].string)
restaurant_numberofreviews


['39 reviews']

In [781]:
restaurant_photos = []
photos = soup8.select(".css-ardur")
restaurant_photos.append(photos[4].string) 
restaurant_photos

['See 64 photos']

In [782]:
restaurant_street = []
restaurant_adress = []
location = soup8.select(".raw__373c0__3rcx7")
restaurant_street.append(location[0].string) 
restaurant_adress.append(location[1].string)
restaurant_street, restaurant_adress

(['1024 S 5th St'], ['Milwaukee, WI 53204'])

In [808]:
restaurant_price = []
price = soup2.select(".css-1xxismk")
price=str(price)
price = re.search('"css-1xxismk">(.+?)<', price)
if price:
    price = price.group(1)
    
price

'$$'

In [806]:
price = re.search('"css-1xxismk">(.+?)<', price)
price

<re.Match object; span=(13, 30), match='"css-1xxismk">$$<'>

In [789]:
restaurant_link = []
link = soup2.select(".css-ac8spe")
restaurant_link.append(link[6].string)
restaurant_link

['read more']

In [790]:
restaurant_rating = []
for item in soup8.select('[class*=container]'):
    if item.find('h4'):
        name = item.find('h4').get_text()
        ratings=soup1.select('[aria-label*=rating]')[0]
        rating=str(ratings)
        rating = re.search('aria-label="(.+?)star rating"', rating)
        rating
        if rating:
            rating = rating.group(1)
            
restaurant_rating.append(rating)

In [791]:
restaurant_rating

['4.5 ']

In [792]:
restaurant_categories = []

#this needs to be inside the loop
categories = []

for item in soup8.select('[class*=container]'):
    if item.find('h1'):
        items=item.select('.css-166la90')
        for i in items:
            categories.append(i.string)
            
        restaurant_categories.append(categories)


In [793]:
restaurant_categories

[['Unclaimed', 'Breweries', 'Beer Bar', 'Beer Gardens']]

In [774]:

    
driver = webdriver.Firefox(options=options) #options=options 
driver.get('https://www.yelp.com/biz/urban-harvest-brewing-company-milwaukee')
            
source = driver.page_source
soup8 = BeautifulSoup(source, 'html.parser')   

price = soup8.select(".claim-text--light__373c0__1ip1u")
price

[<span class="claim-text--light__373c0__1ip1u css-1xxismk"><a class="css-166la90" href="https://biz.yelp.com/signup/coQ9MF9jDBxAbE0Ugq1RKw/account?utm_campaign=claim_business&amp;utm_medium=www&amp;utm_source=biz_page_unclaimed&amp;utm_content=claim_status_link_react&amp;hl=en_US" name="" rel="" target="_blank">Unclaimed</a> <span aria-hidden="true" class="icon--16-exclamation-v2 css-13zy15n"><svg class="icon_svg" height="16" width="16"><path d="M8 1a7 7 0 110 14A7 7 0 018 1zm0 2a5 5 0 100 10A5 5 0 008 3zm0 6.854a1 1 0 011 1.006 1 1 0 01-1 1v-.01a1 1 0 01-.92-.61A1 1 0 018 9.854zM8 4.14a1 1 0 011 1v3a1 1 0 11-2 0v-3a1 1 0 011-1z"></path></svg></span></span>]

In [779]:
updates=soup8.select('.css-gdi06s')
updates[0].string

In [773]:
price = soup2.select(".claim-text--light__373c0__1ip1u")
price

[<span class="claim-text--light__373c0__1ip1u css-df2qti"><span aria-hidden="true" class="icon--16-claim-filled-v2 css-1jnn9ss"><svg class="icon_svg" height="16" width="16"><path clip-rule="evenodd" d="M1 8a7 7 0 1114 0A7 7 0 011 8zm7 2.51l3.58-3.6a1 1 0 10-1.41-1.42l-2.89 2.9-.94-.94a1 1 0 10-1.42 1.41l1.67 1.65a1 1 0 001.41 0z" fill-rule="evenodd"></path></svg></span> <!-- -->Claimed</span>]

In [ ]:
    price=str(price)
    price = re.search('>(.+?)<', price)
    if price:
        price = price.group(1)

In [ ]:
#driver.find_element_by_css_selector('[class=" css-174jypu"]').click()

In [759]:
linking=Links[11:12]
linking

['https://www.yelp.com/biz/movida-hotel-madrid-milwaukee?osq=Restaurants']

In [ ]:
restaurant_about = []

for item in soup1.select('[class*=container]'):
    if item.find('h5'):
        print(item)
        items=item.select('.css-ahxjws')
        print(items[1])


In [ ]:

rating = soup.select(".i-stars__373c0__1T6rz i-stars--large-4-half__373c0__2lYkD border-color--default__373c0__30oMI overflow--hidden__373c0__2B0kz")

In [ ]:
            
            css-gdi06s

            head = driver.find_element_by_css_selector('')
            headline = head.text.strip()
            headline_lst =list(headline.split("\n"))
            type_ = headline_lst[1] 
            name = headline_lst[2]
            beoordeling = headline_lst[3]
        
     #safe the title of the first object
        #title = driver.find_element_by_id('HEADING')
        #title = driver.title()
    #getting source for beautiful soup
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        soup_list = list(soup.stripped_strings)
        catch = soup_list[15:25] #catch all
        category = soup_list[18:19] #category
        title = soup_list[20] #name of museum
        rating = soup_list[21] #rating
        
    
        #soup = soup.getText(',').strip()

        #for string in soup.stripped_strings:
         #   soup_out = repr(string)
        
        driver.quit()  
        
        #for _ in str(soup_out):
         #   soup_outs.append(_)
            
        return type_, name, beoordeling, catch, category, rating, title  
    except (NoSuchElementException, NameError):
        print('element error!')
        driver.quit()
        no_element = 'no'
        return no_element, no_element, no_element, no_element, no_element, no_element, no_element

In [396]:
https://www.yelp.com/biz/odd-duck-milwaukee?osq=Restaurants&start=10
    

'https://www.yelp.com/biz/tusk-milwaukee-2?osq=Restaurants'

In [ ]:
url_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start="

for i in range(0,240,10):
    
    html = requests.get(url_yelp+str(i))
    
    soup = BeautifulSoup(html.content, 'lxml')
    link = soup.select(".css-166la90")
   
    for link in links:
        name_restaurant.append(link.string)
        link_restaurant.append(link.get('href'))

# Making sure we have everything
print(name_restaurant)  
print(link_restaurant)

In [597]:
linklink=Links[6:8]
restrest = Restaurants[6:8]

In [862]:
df5 = pd.DataFrame([],columns=['Restaurant', 'Review'])
df5

,Restaurant,Review


### Now on to the individual reviews

In [ ]:
enum_extract = enumerate(restrest)

def getname():
    key = next(enum_extract)
    return str(key)
restname = getname()

In [599]:
#we initialize a dataframe
df1 = pd.DataFrame([],columns=['Restaurant', 'Review'])


for l in linklink:
    url_yelp = l
    html = requests.get(url_yelp)
    soup2 = BeautifulSoup(html.content, 'lxml')

    ## we search for the number of sites to loop through
    length=soup2.select('.css-e81eai')
    length = str(length)
    #we use a regular expression to filter out the total length of the reviews
    length = re.search('>(\d+) of (\d+)<', length) 
    if length:
        length1 = length.group(1)
        length2 = int(length.group(2))
    
    reviews = []
    for i in range(0,length2*10,10):
        html = requests.get(url_yelp+'&start='+str(i))
        soup3 = BeautifulSoup(html.content, 'lxml')
        soupstring = str(soup3)
        #this is the pattern for all the reviews, it will find all matches so we don't have to worry about not grabbing one. Though we will have to clean this up later
        subsoup = re.findall('"markupDisplayName":(.+?),"voterText":', soupstring)
        returneviews += subsoup
        time.sleep(3)
            
    #outside of the last loop, we create a dict, where we zip the lists, and then we append the dict to the list   
    restname = getname()
    keys = [restname]*len(reviews)
    review_list = list(zip(keys, reviews))

    df2=pd.DataFrame(review_list, columns=['Restaurant','Review'])

    frames= [df1, df2]
    df1 = pd.concat(frames)
       
    time.sleep(3)    

In [600]:
df1['Restaurant']=df1.Restaurant.astype('str')

In [601]:
df1['Review'].str.contains('KC and Shelly S.')

0         NaN
0       False
1       False
2       False
3       False
        ...  
1008    False
1009    False
1010    False
1011    False
1012    False
Name: Review, Length: 1014, dtype: object

In [603]:
df1.to_csv('trial.csv')

In [602]:
df1.Review.nunique()

1014

In [587]:
df1.Review.head()

0                                                    0
0    "KC and Shelly S.","photoSrc":"https://s3-medi...
1    "Hart P.","displayLocation":"Gallatin, TN","al...
2    "Warren D.","displayLocation":"Milwaukee, WI",...
3    "Daniela B.","displayLocation":"Brookfield, WI...
Name: Review, dtype: object

In [ ]:
soup.find_all(class_ = ' margin-b2__373c0__abANL border-color--default__373c0__2oFDT')[1].findChildren(class_ = ' css-1h1j0y3')

In [537]:
df1.Restaurant.nunique()

4

In [538]:
df1.tail()

,Restaurant,Review
699,El Beso Milwaukee - Mexican Restaurante & Cantina,"""Tabitha J."",""displayLocation"":""Slidell, LA"",""..."
700,El Beso Milwaukee - Mexican Restaurante & Cantina,"""Jessica J."",""displayLocation"":""Milwaukee, WI""..."
701,El Beso Milwaukee - Mexican Restaurante & Cantina,"""Christian N."",""displayLocation"":""Brentwood, C..."
702,El Beso Milwaukee - Mexican Restaurante & Cantina,"""Linda B."",""displayLocation"":""Milwaukee, WI"",""..."
703,El Beso Milwaukee - Mexican Restaurante & Cantina,"""Linda B."",""displayLocation"":""Milwaukee, WI"",""..."


In [539]:
frames = [df4, df1]

In [540]:
df4 = pd.concat(frames)
len(df4)

167555

In [556]:
df4.to_csv('all_reviews.csv')

In [518]:
df_preliminary.to_csv('df_prelim.csv', index=False, header=False, encoding='utf8')

In [455]:
len(values)

704

In [554]:
df4.Restaurant.nunique()

209

In [555]:
len(df4)

167555

In [557]:
df4.head(60)

,Restaurant,Review
0,0,0
0,Rice N Roll Bistro,"""Amy S."",""displayLocation"":""Milwaukee, WI"",""al..."
1,Rice N Roll Bistro,"""Suzanne R. W."",""displayLocation"":""Allen, TX"",..."
2,Rice N Roll Bistro,"""Sharon B."",""displayLocation"":""Milwaukee, WI"",..."
3,Rice N Roll Bistro,"""Peter K."",""displayLocation"":""Brookfield, WI"",..."
4,Rice N Roll Bistro,"""John F."",""displayLocation"":""Valders, WI"",""alt..."
5,Rice N Roll Bistro,"""Jen W."",""displayLocation"":""Milwaukee, WI"",""al..."
6,Rice N Roll Bistro,"""Tabitha J."",""displayLocation"":""Slidell, LA"",""..."
7,Rice N Roll Bistro,"""Jessica J."",""displayLocation"":""Milwaukee, WI""..."
8,Rice N Roll Bistro,"""Christian N."",""displayLocation"":""Brentwood, C..."


In [ ]:
df4

In [456]:
values[1]

'"Suzanne R. W.","displayLocation":"Allen, TX","altText":"Suzanne R. W.","userUrl":"/user_details?userid=cBX4WOO0lCGwGt8-QTbqwQ","partnerAlias":null,"friendCount":15,"photoCount":8,"reviewCount":13,"eliteYear":null},"comment":{"text":"Stopped in before the Brewers game and it&amp;#39;s really close to the park. Checked into Yelp and was 60 in line on the waiting list an hour prior. Checked in and waited another 30 mins in person. Plan ahead and it&amp;#39;s worth it.","language":"en"},"localizedDate":"4/18/2021","localizedDateVisited":null,"rating":5,"photos":[{"src":"https://s3-media0.fl.yelpcdn.com/bphoto/a1TqGJiU_YcX_RQ07FxItA/300s.jpg","link":"/biz_photos/mad-rooster-cafe-west-milwaukee?reviewid=TQxlU20iUELxJ3al5Xk9UA&amp;select=a1TqGJiU_YcX_RQ07FxItA","altText":"Photo of Mad Rooster Cafe - West Milwaukee, WI, United States. The Bavarian creme stuffed French toast with a side of sausage. Was really yummy!","width":300,"height":300,"caption":"The Bavarian creme stuffed French toast 